#### Car Evaluation Model

Descrição: O dataset contém diversas informações sobre carros, as classes unacc (unaacceptable), acc (acceptable), good, vgood (very good) e um total de 1728 intâncias

Resultados: O modelo se saiu bem em todas as métricas avaliadas e saiu do treinamento antes do final das épocas pelo early stopping.

In [1]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf


In [8]:

car_evaluation = fetch_ucirepo(id=19)

# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets


In [9]:

encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(X)


In [12]:

le = LabelEncoder()
y_encoded = le.fit_transform(y)
print(np.unique(y))
print(np.unique(y_encoded))


['acc' 'good' 'unacc' 'vgood']
[0 1 2 3]


c:\Users\masar\Documents\UFF\IAVerde\ia_verde\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, validation_split=0.3, callbacks=[early_stop], verbose=1)


c:\Users\masar\Documents\UFF\IAVerde\ia_verde\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4545 - loss: 1.2487 - val_accuracy: 0.6584 - val_loss: 0.8765
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7274 - loss: 0.6750 - val_accuracy: 0.6694 - val_loss: 0.7043
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7617 - loss: 0.5429 - val_accuracy: 0.7851 - val_loss: 0.5562
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8592 - loss: 0.3947 - val_accuracy: 0.8375 - val_loss: 0.4645
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9065 - loss: 0.3025 - val_accuracy: 0.8375 - val_loss: 0.4049
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9157 - loss: 0.2426 - val_accuracy: 0.8485 - val_loss: 0.3425
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9147 - loss: 0.2346 - val_accuracy: 0.8650 - val_loss: 0.3115
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9302 - loss: 0.1809 - val_accuracy: 0.8871 - va

In [14]:
y_pred = model.predict(X_test).argmax(axis=1)
print("Relatório de Classificação:")
print(classification_report(
    y_test, y_pred,
    labels=np.unique(y_test),
    target_names=le.classes_[np.unique(y_test)]
))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Relatório de Classificação:
              precision    recall  f1-score   support

         acc       0.91      0.94      0.93       118
        good       0.80      0.63      0.71        19
       unacc       0.99      0.99      0.99       358
       vgood       0.88      0.88      0.88        24

    accuracy                           0.96       519
   macro avg       0.89      0.86      0.87       519
weighted avg       0.96      0.96      0.96       519

Matriz de Confusão:
[[111   3   3   1]
 [  5  12   0   2]
 [  3   0 355   0]
 [  3   0   0  21]]
